In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

XPATH = By.XPATH

# url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=12&country=Russian&orderType=Desc&sortType=ByYieldToClient'
url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=2400&country=Russian&orderType=Desc&sortType=ByYieldToClient'

In [2]:
class Obligations_parser:
    def __init__(self, driver, new_session = True):
        self.driver = driver
        self.driver.maximize_window()

        if new_session:
            self.driver.get(url)
            
            
        # self.obligations_data = pd.DataFrame(columns = ['Название', "Код", 'Ссылка', "Срок(месяца)", "Цена", "Номинал", "Выплат в год", "Величина купона(-13%)", "Прибыль в год(-13% с купона)", "Доходность(все время)", "Доходность(год)", "Амортизация", 'Рейтинг'])
        self.obligations_data = pd.DataFrame()
    

    def next_layer(self):
        next_page_class = 'Pagination-module__item_ALrvz Pagination-module__item_size_s_voSgR Pagination-module__item_arrow_mCpmW'
        # retry_counter = 0
        
        try:
            self.driver.find_element(XPATH, f'//a[@class = "{next_page_class}"]').click()
            self.elements_iterator()
            return True

        except Exception:
            print('Все страницы пройдены.')

            # self.elements_iterator()
            return(self.obligations_data)
        


    
    def grab_paper_info(self, paper_url : str = ''):
        if paper_url:
            self.driver.execute_script("window.open('');")
            self.driver.switch_to.window(self.driver.window_handles[-1])
            self.driver.get(paper_url)
        
        else:
            self.driver.get(paper_url)

        paper_data = {}

        try:
            paper_data.update({'Название' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__showName_iw6qC"]').text]})
            
            paper_data.update({'Код' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__ticker_j7fZW"]').text]})


            paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
            paper_time = paper_time[paper_time.index('з') + 3 : ]
            paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})
            del paper_time

            paper_data.update({'Цена' : [float(self.driver.find_element(XPATH, '//div[@class = "SecurityInvitingScreen__price_FSP8P"]').text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
            
            paper_data.update({'Рейтинг' : [self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panel_itBzT SecurityHeader__desktop_dL7RD"]/div[@class = "SecurityHeader__panelText_KDJdO"]').text]})

            paper_datas = self.driver.find_elements(XPATH, '//td[@class = "Table-module__cell_RJ0qz"]')



            for counter in range(1, len(paper_datas) + 1, 2):
                match paper_datas[counter - 1].text:
                    case "Величина купона":
                        paper_coupon = float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -2])
                        paper_coupon -= paper_coupon * 0.13     # Вычитаем налог 13% на купоны

                        paper_data.update({'Величина купона(-13%)' : [paper_coupon]})
                    
                    case "Номинал":
                        paper_data.update({'Номинал' : [float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
                    
                    case "Количество выплат в год":
                        if 'Величина купона(-13%)' not in paper_data:
                            paper_data.update({'Величина купона(-13%)' : [0]})
                            
                        paper_data.update({'Выплат в год' : [int(paper_datas[counter].text)]})
                        paper_data.update({'Доходность(год)' : [paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // 2 * paper_data['Доходность(год)'][0]]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                        paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] * 12 / paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                    
                    case 'Амортизация':
                        paper_data.update({'Амортизация' : [True] if paper_datas[counter] == 'Да' else [False]})
                    

            print(paper_data)

            paper_data.update({'Ссылка' : [paper_url]})

            

            # print(pd.DataFrame(paper_data))
            # print(pd.DataFrame.from_dict(paper_data), )

            

            self.obligations_data = pd.concat([self.obligations_data, pd.DataFrame(paper_data)])

            
            self.driver.close()

        except Exception:
            self.driver.close()
            time.sleep(10)
            self.grab_paper_info()

    def elements_iterator(self):

        link_buttons_class = 'Link-module__link_yQVl1 Link-module__link_theme_default_mkRhf'

        # Использую тр и тд, поскольку на сайте в строках тр 5 ссылок с нужным адресом, потому просто по классу ссылки не выйдет найти одно совпадение - придется потом чистить список. Приходится потихоьнку находить элементы.
        tr_link_buttons_class = 'Table-module__row_Qlwsh Table-module__row_clickable_FeO1O'
        # td_link_buttons_class = 'Table-module__cell_RJ0qz'

        papers_rows = self.driver.find_elements(XPATH, f'//tr[@class = "{tr_link_buttons_class}"]')
        
        for paper in papers_rows:
            self.driver.switch_to.window(self.driver.window_handles[-1])

            paper_link = paper.find_element(XPATH, 'td').find_element(XPATH, 'a').get_attribute('href')

            
            self.grab_paper_info(paper_link)
            print(len(self.obligations_data), '\n')

            

    
        return(self.obligations_data)

            

        # self.next_layer()



        # for paper in papers_rows:
        #     paper = paper.find_element(XPATH, '//td').find_element(XPATH, f'//a[@class = "{link_buttons_class}"]')
        #     print(paper.get_attribute('href'))




In [3]:
a = webdriver.Chrome()

In [4]:
parser_object = Obligations_parser(a, True)

In [5]:
parser_object.elements_iterator()

{'Название': ['Ferroni выпуск 1'], 'Код': ['RU000A103XP8'], 'Срок(месяца)': [24], 'Цена': [750.0], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [9.222], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [110.66399999999999], 'Доходность(все время)': [221.32799999999997]}
1 

{'Название': ['Феррони БО-01'], 'Код': ['RU000A1053R3'], 'Срок(месяца)': [22], 'Цена': [933.2], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [11.745], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [140.94], 'Доходность(все время)': [258.39]}
2 

{'Название': ['СПМК БО-02'], 'Код': ['RU000A106FN3'], 'Срок(месяца)': [5], 'Цена': [966.8], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [30.884999999999998], 'Номинал': [1000.0], 'Выплат в год': [4], 'Доходность(год)': [123.53999999999999], 'Доходность(все время)': [463.275]}
3 

{'Название': ['Агрофирма-племзавод Победа выпуск 1'], 'Код': ['RU000A102S23'], 'Срок(месяца)': [4], 'Цена': [939.3], 'Рейтинг': ['Низкий'], 'Величина купона(-1

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.92)
Stacktrace:
	GetHandleVerifier [0x00007FF67E537892+54818]
	(No symbol) [0x00007FF67E4A6AC2]
	(No symbol) [0x00007FF67E35DA3B]
	(No symbol) [0x00007FF67E33DEE8]
	(No symbol) [0x00007FF67E3C3237]
	(No symbol) [0x00007FF67E3D6C6F]
	(No symbol) [0x00007FF67E3BE883]
	(No symbol) [0x00007FF67E393691]
	(No symbol) [0x00007FF67E3948D4]
	GetHandleVerifier [0x00007FF67E89B992+3610402]
	GetHandleVerifier [0x00007FF67E8F1860+3962352]
	GetHandleVerifier [0x00007FF67E8E9D4F+3930847]
	GetHandleVerifier [0x00007FF67E5D3646+693206]
	(No symbol) [0x00007FF67E4B1628]
	(No symbol) [0x00007FF67E4AD934]
	(No symbol) [0x00007FF67E4ADA62]
	(No symbol) [0x00007FF67E49E113]
	BaseThreadInitThunk [0x00007FF9F1477344+20]
	RtlUserThreadStart [0x00007FF9F1D426B1+33]


Ошибка     
49 paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
---> 50 paper_time = paper_time[paper_time.index('з') + 3 : ]
     51 paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})

ValueError: substring not found

на https://www.tinkoff.ru/invest/bonds/RU000A106NT4/

In [10]:
df = parser_object.obligations_data.reset_index().drop('index', axis = 1)

In [11]:
df

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,Ferroni выпуск 1,RU000A103XP8,24,750.0,Низкий,9.222,1000.0,12.0,110.664,221.328,https://www.tinkoff.ru/invest/bonds/RU000A103XP8/
1,Феррони БО-01,RU000A1053R3,22,933.2,Низкий,11.745,1000.0,12.0,140.940,258.390,https://www.tinkoff.ru/invest/bonds/RU000A1053R3/
2,СПМК БО-02,RU000A106FN3,5,966.8,Низкий,30.885,1000.0,4.0,123.540,463.275,https://www.tinkoff.ru/invest/bonds/RU000A106FN3/
3,Агрофирма-племзавод Победа выпуск 1,RU000A102S23,4,939.3,Низкий,21.141,1000.0,4.0,84.564,253.692,https://www.tinkoff.ru/invest/bonds/RU000A102S23/
4,Alfa Bank,RU000A100GW5,11,8350000.0,Средний,254862.150,10000000.0,4.0,1019448.600,8410450.950,https://www.tinkoff.ru/invest/bonds/RU000A100GW5/
...,...,...,...,...,...,...,...,...,...,...,...
809,РЖД 1Р-26R,RU000A106K43,57,1011.4,Высокий,9.396,1000.0,12.0,112.752,535.572,https://www.tinkoff.ru/invest/bonds/RU000A106K43/
810,Альфа-Банк 002Р выпуск 14,RU000A103RS4,2,1002.0,Высокий,34.452,1000.0,2.0,68.904,413.424,https://www.tinkoff.ru/invest/bonds/RU000A103RS4/
811,Сибнефтехимтрейд выпуск 2,RU000A104DZ7,4,1013.9,Низкий,13.224,1000.0,12.0,158.688,52.896,https://www.tinkoff.ru/invest/bonds/RU000A104DZ7/
812,Истрсыр БО,RU000A105ZF9,2,1051.0,Низкий,12.789,1000.0,12.0,153.468,25.578,https://www.tinkoff.ru/invest/bonds/RU000A105ZF9/


In [13]:
df.to_csv('Obligations_data_parsed.csv')

In [ ]:
df[df['Номинал'] <= 15000].sort_values(by=['Доходность(год)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,ПЮДМ БО-П2,RU000A1020K7,21,10000.00,Средний,92.916,10000.0,12,1114.992,92.916,https://www.tinkoff.ru/invest/bonds/RU000A1020K7/
0,АПРИ Флай Плэнинг,RU000A105DS9,37,1011.20,Средний,52.026,1000.0,4,208.104,468.234,https://www.tinkoff.ru/invest/bonds/RU000A105DS9/
0,АПРИ Флай Плэнинг выпуск 3,RU000A106WZ2,47,1020.00,Средний,45.501,1000.0,4,182.004,500.511,https://www.tinkoff.ru/invest/bonds/RU000A106WZ2/
0,Завод КЭС выпуск 3,RU000A105AW7,36,1130.00,Средний,43.326,1000.0,4,173.304,389.934,https://www.tinkoff.ru/invest/bonds/RU000A105AW7/
0,Завод КЭС выпуск 4,RU000A106QS9,58,957.20,Низкий,42.108,1000.0,4,168.432,589.512,https://www.tinkoff.ru/invest/bonds/RU000A106QS9/
...,...,...,...,...,...,...,...,...,...,...,...
0,ГК Пионер-001P выпуск 4,RU000A1016F5,2,174.37,Средний,3.915,175.0,4,15.660,0.000,https://www.tinkoff.ru/invest/bonds/RU000A1016F5/
0,Пионер-Лизинг-БО-П02,RU000A1006C3,63,1060.40,Низкий,0.870,1000.0,12,10.440,4.350,https://www.tinkoff.ru/invest/bonds/RU000A1006C3/
0,Верхнебаканский цементный завод 001P выпуск 1,RU000A0ZYLK7,50,973.90,Средний,1.566,1000.0,4,6.264,18.792,https://www.tinkoff.ru/invest/bonds/RU000A0ZYLK7/
0,ПКТ выпуск 2,RU000A0JW5E3,28,755.20,Высокий,0.000,1000.0,2,0.000,0.000,https://www.tinkoff.ru/invest/bonds/RU000A0JW5E3/


In [ ]:
df[df['Код'] == 'RU000A106WZ2']

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,АПРИ Флай Плэнинг выпуск 3,RU000A106WZ2,47,102.0,Средний,45.501,1000.0,4,182.004,500.511,https://www.tinkoff.ru/invest/bonds/RU000A106WZ2/
